<div>
    <div style="float:left;">
        <img src="https://root.cern.ch/root/htmldoc/guides/users-guide/pictures/rootlogo.png" width="400px" />
    </div>
    <div style="float:left;">
        <img src="https://rawgit.com/qati/GSOC16/master/img/tmva-logo.svg" width="500px"/>
    </div>
</div>

<hr style="border-top-width: 4px; border-top-color: #34609b;">

<!--<script src="JsRoot/scripts/JSRootCore.js?jq2d&onload=JsRootLoadedCall" type="text/javascript"></script>-->

In [1]:
import ROOT
from ROOT import TFile, TMVA, TCut

Welcome to JupyROOT 6.09/01


## Enable JS visualization

In [2]:
%jsmva on

# Dataset infos

In [3]:
infname     = "tmva_class_example.root"
dataset     = "tmva_class_example"
treeNameSig = "TreeS"
treeNameBkg = "TreeB"
outfname    = "TMVA.root"

## Declare Factory and DataLoader

In [5]:
outputFile = TFile( outfname, 'RECREATE' )

TMVA.Tools.Instance();

In [6]:
factory = TMVA.Factory(JobName="TMVAClassification", TargetFile=outputFile,
            V=False, Transformations=["I", "D", "P", "G","D"], AnalysisType="Classification")

In [7]:
loader = TMVA.DataLoader(dataset)

## Adding variables to DataLoader

In [8]:
loader.AddVariable( "myvar1 := var1+var2", 'F' )
loader.AddVariable( "myvar2 := var1-var2", "Expression 2", 'F' )
loader.AddVariable( "var3",                "Variable 3", 'F' )
loader.AddVariable( "var4",                "Variable 4", 'F' )

loader.AddSpectator( "spec1:=var1*2",  "Spectator 1",  'F' )
loader.AddSpectator( "spec2:=var1*3",  "Spectator 2",  'F' )

## Setting up dataset from Trees

In [11]:
input = TFile.Open( infname )

signal      = input.Get( treeNameSig )
background  = input.Get( treeNameBkg )

signalWeight     = 1.0
backgroundWeight = 1.0

mycuts = TCut("")
mycutb = TCut("")

loader.AddSignalTree(signal, signalWeight)
loader.AddBackgroundTree(background, backgroundWeight)

loader.PrepareTrainingAndTestTree(SigCut=mycuts, BkgCut=mycutb,
            nTrain_Signal=1000, nTrain_Background=1000, nTest_Signal=2000, nTest_Background=2000,
                                  SplitMode="Random", NormMode="NumEvents", V=False);

## Visualizing input variables

In [12]:
loader.DrawInputVariable("myvar1")

### We can also visualize transformations on input variables

In [13]:
loader.DrawInputVariable("myvar1", processTrfs=["I", "D", "P", "G","D"]) #I;N;D;P;U;G,D

## Correlation matrix of input variables

In [14]:
loader.DrawCorrelationMatrix("Signal")

## Booking DNN

In [15]:
factory.BookDNN(loader)

## Booking other methods

In [16]:
factory.BookMethod( loader,TMVA.Types.kMLP, "MLP", 
        H=False, V=False, NeuronType="tanh", VarTransform="N", NCycles=600, HiddenLayers="N+5",
                   TestRate=5, UseRegulator=False )

factory.BookMethod( loader, TMVA.Types.kBDT, "BDT",
    H=False, V=False, NTrees=850, MinNodeSize="2.5%", MaxDepth=3, BoostType="AdaBoost", AdaBoostBeta=0.5,
                   UseBaggedBoost=True, BaggedSampleFraction=0.5, SeparationType="GiniIndex", nCuts=20 )

<ROOT.TMVA::MethodBDT object ("BDT") at 0x7950150>

# Train Methods

In [17]:
factory.TrainAllMethods()

TFHandler_DNN
VariableMeanRMSMinMax
myvar10.0330140.38087-1.00001.0000
myvar20.0777380.30037-1.00001.0000
var30.113290.37003-1.00001.0000
var40.131010.42584-1.00001.0000
TFHandler_DNN
VariableMeanRMSMinMax
myvar10.0330140.38087-1.00001.0000
myvar20.0777380.30037-1.00001.0000
var30.113290.37003-1.00001.0000


## Testing the methods

In [18]:
factory.TestAllMethods()

## Evaluate the methods

In [19]:
factory.EvaluateAllMethods()

## Classifier Output Distributions

In [20]:
factory.DrawOutputDistribution(dataset, "MLP")

## ROC curve

In [24]:
factory.DrawROCCurve(dataset)

## Classifier Cut Efficiencies

In [25]:
factory.DrawCutEfficiencies(dataset, "MLP")

## Draw Neural Network

* Mouseover (node, weight): focusing
* Zooming and grab and move supported
* Reset: double click

In [26]:
factory.DrawNeuralNetwork(dataset, "MLP")

## Draw Deep Neural Network

In [27]:
factory.DrawNeuralNetwork(dataset, "DNN")

## Draw Decision Tree

* Mouseover (node, weight): showing decision path
* Zooming and grab and move supported
* Reset: double click
* Click on node: 
    * hiding subtree, if node children are hidden the node will have a green border
    * rescaling: bigger nodes, bigger texts
    * click again to show the subtree

In [28]:
factory.DrawDecisionTree(dataset, "BDT") #11

## DNN weights heat map

In [29]:
factory.DrawDNNWeights(dataset, "DNN")

## Close the factory's output file

In [30]:
outputFile.Close()